In [1]:
import requests
import json
import time
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [2]:
username = ""
password = ""

contest_id = "rco-contest-2019-qual"
problem_id = "rco_contest_2019_qual_b"

contest_start = datetime.datetime(2023,1,13,19,0,0)
contest_end = datetime.datetime(2023,1,22,19,0,0)

In [7]:
personal = {
 'utprg': ['benchmark'],
 'milkcoffee': ['山走暁飛'],
 'akane831': ['輝かしい未来を目指す不屈の戦い人'],
 'otsuneko': ['>euphonic'],
 'nephrologist': ['benchmark'],
 'bio4eta': ['霧ヶ峰めっちゃ良い買い物しました！！！！'],
 'keizin': ['輝かしい未来を目指す不屈の戦い人'],
 'nok0': ['山走暁飛'],
 'yuuki_n': ['逆襲のバベルクライマーズ'],
 'jaku': ['輝かしい未来を目指す不屈の戦い人'],
 'tnodino': ['霧ヶ峰めっちゃ良い買い物しました！！！！'],
 'ainem': ['運営'],
 'burioden': ['AHCやってみ隊'],
 'r3yohei': ['benchmark'],
 'hal0taso': ['霧ヶ峰めっちゃ良い買い物しました！！！！'],
 'keroru': ['輝かしい未来を目指す不屈の戦い人'],
 'motoshira': ['逆襲のバベルクライマーズ'],
 'Gex777': ['>euphonic'],
 'cplusplusonly': ['>euphonic'],
 'highjump': ['山走暁飛'],
 'fujikawahiroaki': ['逆襲のバベルクライマーズ'],
 'kokatsu': ['逆襲のバベルクライマーズ'],
 'brthyyjp': ['benchmark'],
 'zatt': ['霧ヶ峰めっちゃ良い買い物しました！！！！'],
 'hiraku': ['>euphonic'],
 '8wings': ['山走暁飛'],
 'gunmaneko': ['AHCやってみ隊'],
 'jamuojisan': ['AHCやってみ隊'],
 'michirakara': ['霧ヶ峰めっちゃ良い買い物しました！！！！'],
 'uni_kakurenbo': ['>euphonic']
 }

In [8]:
for name in personal.keys():
  url = "https://atcoder.jp/users/" + name + "/history/json"
  json = requests.session().get(url).json()
  argo_rate = 0
  for data in json:
    ymd, hms_ = data["EndTime"].split("T")
    y,m,d = map(int,ymd.split("-"))
    h,min,sec = map(int,hms_.split("+")[0].split(":"))
    end_time = datetime.datetime(y,m,d,h,min,sec)
    if end_time > contest_start:
      break
    argo_rate = data["NewRating"]

  url = "https://atcoder.jp/users/" + name + "/history/json?contestType=heuristic"
  json = requests.session().get(url).json()
  heur_rate = 0
  heur_highest_perf = 0
  for data in json:
    ymd, hms_ = data["EndTime"].split("T")
    y,m,d = map(int,ymd.split("-"))
    h,min,sec = map(int,hms_.split("+")[0].split(":"))
    end_time = datetime.datetime(y,m,d,h,min,sec)
    if end_time > contest_start:
      break
    heur_rate = data["NewRating"]
    heur_highest_perf = max(heur_highest_perf, data["NewRating"])
  personal[name] += [argo_rate, heur_rate, heur_highest_perf]


In [9]:
df=pd.DataFrame.from_dict(personal, orient="index", columns=["チーム名","Aレート","Hレート","BestPerf"])
df.to_csv("personal_info.csv")

In [10]:
personal = pd.read_csv("personal_info.csv", index_col=0)
personal.head()

,チーム名,Aレート,Hレート,BestPerf
utprg,benchmark,349,418,418
milkcoffee,山走暁飛,2057,1733,1733
akane831,輝かしい未来を目指す不屈の戦い人,916,1086,1086
otsuneko,>euphonic,892,1483,1483
nephrologist,benchmark,1655,1325,1325


In [ ]:
# User Agentの設定の設定
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4.1 Safari/605.1.15"
header = {
    'User-Agent': user_agent
}

login_url = "https://atcoder.jp/login?continue=https%3A%2F%2Fatcoder.jp%2F"
session = requests.session()
response = session.get(login_url, headers=header)

# BeautifulSoupオブジェクト作成(token取得の為)
bs = BeautifulSoup(response.text, 'html.parser')
# tokenの取得
csrf_token = bs.find(attrs={'name':'csrf_token'}).get('value')

login_data = {
   'username': username,
   'password': password,
   'csrf_token': csrf_token,
}

# cookieの取得
response_cookie = response.cookies

login = session.post(login_url, data=login_data, cookies=response_cookie)
time.sleep(2) # 少し時間を置いてみる

In [ ]:
standings_url = "https://atcoder.jp/contests/" + contest_id + "/standings/json"
standings_json = session.get(standings_url).json()

In [ ]:
def make_datetime(s: str) -> datetime:
  ymd, hms = s.split(" ")
  year, month, day = map(int,ymd.split("-"))
  hour, minute, second = hms.split(":")
  second = int(second.split("+")[0])

  res = datetime.datetime(year=year, month=month, day=day, hour=int(hour), minute=int(minute), second=int(second))

  return res

In [ ]:
records = {
    "name": [],
    "team": [],
    "lang": [],
    "score": [],
    "submit": [],
    "elapsed_time": [],
    #"argo_rate": [],
    #"heuristic_rate": [],
    #"performance": []
}

submission_url = "https://atcoder.jp/contests/" + contest_id +"/submissions?desc=true&f.LanguageName=&f.Status=AC&f.Task=" + problem_id + "&f.User=&orderBy=created&page="
page=1

not_in_record = set()

while True:
  submissions = session.get(submission_url + str(page))
  soup = BeautifulSoup(submissions.text, "html5lib")
  tr = soup.select("tbody > tr")
  if len(tr)==0:
    break
  for row in tr:
    submit = make_datetime(row.contents[1].text)
    if contest_start < submit < contest_end:
      name = row.contents[5].text.rstrip() if " " in row.contents[5].text else row.contents[5].text
      lang = row.contents[7].text
      score = int(row.find(name="td", attrs={"class": "submission-score"}).text)
      elapsed_time = row.contents[14].text
      if name in personal.index:
        tname = personal.at[name, "チーム名"]
        records["name"].append(name)
        records["team"].append(tname)
        records["lang"].append(lang)
        records["score"].append(score)
        records["submit"].append(submit)
        records["elapsed_time"].append(elapsed_time)
        #records["argo_rate"].append(a_rate)
        #records["heuristic_rate"].append(h_rate)
        #records["performance"].append(perf)
      else:
        not_in_record.add(name)
    elif submit < contest_start:
      break
  else:
    page += 1
    continue
  break

if not_in_record: print("バチャ期間中に提出があったが、参加者に登録されていない：", not_in_record)

バチャ期間中に提出があったが、参加者に登録されていない： {'gengar_094', 'yoshiaki_kkk', 'Klavis'}


In [ ]:
df = pd.DataFrame(data = records)
df

,name,team,lang,score,submit,elapsed_time
0,r3yohei,benchmark,Rust (1.42.0),607852,2023-01-22 18:55:38,1991 ms
1,brthyyjp,benchmark,PyPy3 (7.3.0),595747,2023-01-22 18:55:27,1901 ms
2,fujikawahiroaki,逆襲のバベルクライマーズ,Crystal (0.33.0),565300,2023-01-22 18:51:48,1963 ms
3,r3yohei,benchmark,Rust (1.42.0),611391,2023-01-22 18:50:36,1991 ms
4,brthyyjp,benchmark,PyPy3 (7.3.0),597179,2023-01-22 18:50:14,1900 ms
...,...,...,...,...,...,...
438,nok0,山走暁飛,C++ (GCC 9.2.1),480411,2023-01-13 19:34:47,25 ms
439,tnodino,霧ヶ峰めっちゃ良い買い物しました！！！！,Python (3.8.2),214824,2023-01-13 19:31:52,29 ms
440,tnodino,霧ヶ峰めっちゃ良い買い物しました！！！！,Python (3.8.2),250710,2023-01-13 19:26:51,28 ms
441,8wings,山走暁飛,PyPy3 (7.3.0),285183,2023-01-13 19:20:46,142 ms


In [ ]:
df.to_csv("submissions.csv")